In [35]:
"""
Scratch notebook to build as we go. All functionality folded back into
relevant python files.
"""

'\nScratch notebook to build as we go. All functionality folded back into\nrelevant python files.\n'

In [36]:
# download, split, tokenize, and save the training and test data
!python3 ./data/prepare.py

Loading dataset shards: 100%|███████████████████| 80/80 [00:01<00:00, 46.35it/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 7973700
    })
    val: Dataset({
        features: ['text'],
        num_rows: 40069
    })
})

writing /Users/vinay/src/gpt-2/./data/train.bin:  12%| | 127/1024 [00:14<02:04, ^C
writing /Users/vinay/src/gpt-2/./data/train.bin:  12%|▏| 128/1024 [00:14<01:41, 
Traceback (most recent call last):
  File "/Users/vinay/src/gpt-2/./data/prepare.py", line 55, in <module>
    arr_batch = np.concatenate(batch['ids'])
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


In [64]:
import os

import numpy as np
import tiktoken

""" let's build a decoder to validate we can take the prepared data back into text """
enc = tiktoken.get_encoding("gpt2")

train_data_bin = os.path.join('data', 'train.bin')
test_data_bin = os.path.join('data', 'test.bin')

# filename is a path object to either the train or test data bins
# num_docs is the number of documents you want to read from the file
def load_data(filename, num_docs = 1):
    docs = [] # array of docs in token format
    current_doc = np.zeros((0,), np.uint16)
    data_offset = 0 # where we are in the file
    data_read_count = 1_000_000 # how many tokens to read at a time

    # note each item has a size of 2 bytes (16-bits)
    file_stats = os.stat(filename)
    file_size = file_stats.st_size
    total_items = file_size / 2

    data_arr = np.memmap(filename, dtype=np.uint16, mode='r')

    while data_offset < total_items and len(docs) < num_docs:
        data_arr = np.memmap(filename, dtype=np.uint16, mode='r')
        data = data_arr[data_offset:data_offset + data_read_count]
        data_offset += len(data)
        for id in data:
            current_doc = np.append(current_doc, id)
            if id == enc.eot_token:
                docs.append(current_doc)
                current_doc = np.zeros((0,), np.uint16)
            if len(docs) == num_docs:
                break
    
    return docs

In [67]:
data_samples = load_data(train_data_bin)
for sample in data_samples:
    print(enc.decode(sample))

'Addressing Retailer Concerns, Part 1'

'ASM' #25 -- 9.99 cover price

The first afternoon of the Marvel Retailer Summit (see " Marvel Retailer Summit ") was a free-flowing discussion of retailer concerns, led by Senior Vice President – Sales and Marketing David Gabriel and Editor-in-Chief Axel Alonso. In Part 1 of this two-part article, we cover the discussion of the many changes to Marvel characters over the past few years, reboots and restarts, and expanding high-selling franchise titles. In Part 2 , we covered the discussion of talent management, securing hot talent, creating new talent, event fatigue, timing of events, and trade pricing. At the end of the event, we also asked David Gabriel some questions about the shift in the market last fall that motivated, in part, the Retailer Summit (see " Marvel’s David Gabriel on the 2016 Market Shift ").The first topic that came up at the Summit, although it wasn’t on the agenda in that form, was the number of changes that had been made to